In [ ]:
#| default_exp ai

# 🧠 AI-Powered Text Editing

This module provides structured text editing using OpenAI's API with Pydantic validation.

**Features:**
- Structured outputs via Pydantic models ensuring type-safe operations
- Iterative editing with full context preservation
- Efficient conversation management (single assistant message + cumulative user instructions)
- Natural language support for commands like "change it back" or "undo that"

In [ ]:
#| export
from __future__ import annotations
from typing import List, Dict, Literal, Union, Optional, Any
from pydantic import BaseModel, ConfigDict, Field, model_validator
from openai import OpenAI
from dotenv import load_dotenv
import os
import re
import subprocess
import json
import requests
import platform

class OllamaManager:
    """Manages Ollama installation and model operations."""

    def __init__(self):
        self.base_url = "http://localhost:11434"

    def is_installed(self) -> bool:
        """Check if Ollama is installed and accessible."""
        try:
            result = subprocess.run(['ollama', '--version'],
                                    capture_output=True, text=True, timeout=5)
            return result.returncode == 0
        except (subprocess.TimeoutExpired, FileNotFoundError):
            return False
    
    def is_running(self) -> bool:
        """Check if Ollama service is running."""
        try:
            response = requests.get(f"{self.base_url}/api/tags", timeout=5)
            return response.status_code == 200
        except requests.RequestException:
            return False
    
    def get_install_instructions(self) -> str:
        """Get platform-specific installation instructions."""
        system = platform.system().lower()
        if system == "linux":
            return "curl -fsSL https://ollama.ai/install.sh | sh"
        elif system == "darwin":  # macOS
            return "Download from https://ollama.ai/download or use: brew install ollama"
        elif system == "windows":
            return "Download from https://ollama.ai/download"
        else:
            return "Visit https://ollama.ai/download for installation instructions"
        
    def list_models(self) -> List[Dict[str, Any]]:
        """List all installed Ollama models."""
        if not self.is_running():
            return []
        
        try:
            response = requests.get(f"{self.base_url}/api/tags", timeout=10)
            if response.status_code == 200:
                data = response.json()
                return data.get("models", [])
        except requests.RequestException:
            pass
        return []
    
    def pull_model(self, model_name: str) -> bool:
        """Download/pull a model using Ollama."""
        try:
            result = subprocess.run(['ollama', 'pull', model_name],
                                    capture_output=True, text=True, timeout=300)
            return result.returncode == 0
        except (subprocess.TimeoutExpired, FileNotFoundError):
            return False
    
    def get_popular_models(self) -> List[Dict[str, str]]:
        """Get a list of popular models for easy installation."""
        return [
            {"name": "llama3.2", "description": "Latest Llama model (3B params)"},
            {"name": "llama3.2:1b", "description": "Smaller Llama model (1B params)"},
            {"name": "gemma2", "description": "Google's Gemma model"},
            {"name": "phi3", "description": "Microsoft's Phi-3 model"},
            {"name": "mistral", "description": "Mistral AI model"},
            {"name": "codellama", "description": "Code-specialized Llama model"},
            {"name": "qwen2.5", "description": "Alibaba's Qwen model"},
        ]

# Global Ollama manager instance
_ollama_manager = OllamaManager()

def get_ollama_status() -> Dict[str, Any]:
    """Get comprehensive Ollama status information."""
    return {
        "installed": _ollama_manager.is_installed(),
        "running": _ollama_manager.is_running(),
        "install_instructions": _ollama_manager.get_install_instructions(),
        "models": _ollama_manager.list_models(),
        "popular_models": _ollama_manager.get_popular_models()
    }

def setup_ollama_client(model_name: str = "llama3.2") -> bool:
    """Setup OpenAI client to use Ollama with specified model."""
    global _client, _current_model

    if not _ollama_manager.is_running():
        return False
    
    try:
        _client = OpenAI(
            base_url='http://localhost:11434/v1/',
            api_key='ollama'  # Ollama doesn't need a real API key
        )
        _current_model = model_name
        return True
    except Exception:
        return False

def setup_openai_client(api_key: str, model_name: str = "gpt-4o-mini") -> bool:
    """Setup OpenAI client to use OpenAI API."""
    global _client, _current_model
    
    try:
        _client = OpenAI(api_key=api_key)
        _current_model = model_name
        return True
    except Exception:
        return False

# Global variables to track current setup
_client = None
_current_model = ""

# Try to auto-setup on import
def _auto_setup():
    """Try to automatically setup a client on import."""
    global _client

    # Try OpenAI first
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    if api_key and setup_openai_client(api_key):
        return
    
    # Fall back to Ollama
    if _ollama_manager.is_running():
        setup_ollama_client()

_auto_setup()

# 🧩 Edit Operation System Overview

- This system defines a structured edit language for transforming text.
- Each edit is represented as a Pydantic model, ensuring type safety, validation, and predictable behavior.

### ⚙️ Core Concept

The system revolves around three layers:
1. `Targeting` – defines where an operation applies.
2. **Operation models** – define what kind of edit to perform.
3. **Executor functions** (`_apply_plan`, `_apply_in_scope`, etc.) – perform the actual modifications.

## 🎯 Targeting (Scope and Selection)

**Targeting model:**
_Almost every edit benefits from a lightweight targeting system to not explode the number of ops._

`Targeting` fields:
| Field                         | Type             | Description                                                                     |
| ----------------------------- | ---------------- | ------------------------------------------------------------------------------- |
| `ignore_case`                 | `bool`           | Case-insensitive matching (default: `False`)                                    |
| `nth`                         | `int` | `"last"` | Apply operation only to the nth or last match                                   |
| `all`                         | `bool`           | Apply to **all** matches in the scope (overrides `nth`)                         |
| `scope`                       | `Scope`          | Limits where edits occur (`global`, `line`, `line_range`, or `between_markers`) |
| `line_index`                  | `int`            | Target a single line (when `scope="line"`)                                      |
| `line_start` / `line_end`     | `int`            | Inclusive/exclusive range of lines (when `scope="line_range"`)                  |
| `start_marker` / `end_marker` | `str`            | Marker-based boundaries (when `scope="between_markers"`)                        |
| `include_markers`             | `bool`           | Whether markers themselves are included in the scoped region                    |

Validation ensures that each `scope` provides the required fields.
For example, `scope="line"` requires `line_index`.

### 🧱 Operation Types

- Each edit operation is represented as its own Pydantic model. 
- Every operation has an `op` field identifying its type and an optional `target` (the `Targeting` object).

### 📝 Replacement Operations
| Class                | Description                                         |
| -------------------- | --------------------------------------------------- |
| **`ReplaceAllOp`**   | Replace all exact occurrences (no regex).           |
| **`ReplaceFirstOp`** | Replace only the first match.                       |
| **`RegexReplaceOp`** | Regex-based replacement with backrefs (`\1`, `\2`). |

### ➕ Insertion Operations
| Class                | Description                                        |
| -------------------- | -------------------------------------------------- |
| **`InsertAtOp`**     | Insert text at a given character index.            |
| **`InsertAfterOp`**  | Insert after the first/last/nth match of a string. |
| **`InsertBeforeOp`** | Insert before a given match.                       |
| **`PrependOp`**      | Add text at the beginning (or scoped start).       |
| **`AppendOp`**       | Add text at the end (or scoped end).               |

### ❌ Deletion Operations
| Class                 | Description                                                 |
| --------------------- | ----------------------------------------------------------- |
| **`DeleteOp`**        | Delete exact text (supports nth/all/ignore_case).           |
| **`DeleteRegexOp`**   | Delete text matching a regex pattern.                       |
| **`DeleteRangeOp`**   | Delete text between absolute character indices.             |
| **`DeleteBetweenOp`** | Delete text between markers (optionally including markers). |

### ✂️ Structural Operations
| Class                       | Description                                                      |
| --------------------------- | ---------------------------------------------------------------- |
| **`WrapWithOp`**            | Wrap a literal in prefix/suffix (e.g., add quotes or HTML tags). |
| **`NormalizeWhitespaceOp`** | Simplify whitespace, remove extra newlines/spaces.               |

### 📄 Line-based Operations
These make voice-based or code edits more natural:
| Class                    | Description                                  |
| ------------------------ | -------------------------------------------- |
| **`SetLineTextOp`**      | Replace text of a specific line.             |
| **`InsertLineBeforeOp`** | Insert a new line before a given line index. |
| **`InsertLineAfterOp`**  | Insert a new line after a given line index.  |
| **`DeleteLineOp`**       | Delete a specific line by index.             |


In [ ]:
#| export

Scope = Literal["global", "line", "line_range", "between_markers"]

class Targeting(BaseModel):
    """Optional targeting/scope for an operation."""
    ignore_case: bool = False
    # which match to touch (if not 'all'): 1 = first, 2 = second, "last" = last
    nth: Optional[Union[int, Literal["last"]]] = None
    # if true, apply to all matches in the selected scope (overrides nth)
    all: bool = False

    # scope of operation
    scope: Scope = "global"

    # line scope
    line_index: Optional[int] = Field(None, ge=0, description="0-based line index for scope='line'")
    line_start: Optional[int] = Field(None, ge=0, description="inclusive; for scope='line_range'")
    line_end: Optional[int] = Field(None, ge=0, description="exclusive; for scope='line_range'")

    # marker scope
    start_marker: Optional[str] = None
    end_marker: Optional[str] = None
    include_markers: bool = False  # only relevant for scope selection; not the DeleteBetween op

    model_config = ConfigDict(extra="forbid")

    @model_validator(mode="after")
    def _validate(self):
        if self.scope == "line" and self.line_index is None:
            raise ValueError("scope='line' requires line_index")
        if self.scope == "line_range" and (self.line_start is None or self.line_end is None):
            raise ValueError("scope='line_range' requires line_start and line_end")
        if self.scope == "line_range" and self.line_end is not None and self.line_start is not None:
            if self.line_end < self.line_start:
                raise ValueError("line_end must be >= line_start")
        if self.scope == "between_markers" and (not self.start_marker or not self.end_marker):
            raise ValueError("scope='between_markers' requires start_marker and end_marker")
        if isinstance(self.nth, int) and self.nth < 1:
            raise ValueError("nth must be >= 1 if provided")
        return self

# ───────────────────────────────────────────────────────────────────────────────
# Existing basic ops (kept; minor enhancements to add ignore_case where useful)
# ───────────────────────────────────────────────────────────────────────────────

class ReplaceAllOp(BaseModel):
    """Exact literal replacement (no regex)."""
    op: Literal["replace_all"]
    find: str = Field(..., min_length=1)
    replace: str = Field(..., min_length=0)
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class RegexReplaceOp(BaseModel):
    """Regex-based replacement with backrefs like \\1, \\2."""
    op: Literal["regex_replace"]
    pattern: str = Field(..., min_length=1)
    replacement: str = Field(..., min_length=0)
    flags_ignore_case: bool = False
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")
    @model_validator(mode="after")
    def _validate_regex(cls, v: "RegexReplaceOp"):
        try:
            re.compile(v.pattern)
        except re.error as e:
            raise ValueError(f"Invalid regex pattern: {e}") from e
        return v

class InsertAtOp(BaseModel):
    """Insert text at an absolute character index (0-indexed) in the (scoped) text."""
    op: Literal["insert_at"]
    text: str = Field(..., min_length=1)
    position: int = Field(..., ge=0)
    target: Optional[Targeting] = None  # scope defines where 'position' is measured
    model_config = ConfigDict(extra="forbid")

class InsertAfterOp(BaseModel):
    """Insert text after the first match of 'after' in the (scoped) text."""
    op: Literal["insert_after"]
    text: str = Field(..., min_length=1)
    after: str = Field(..., min_length=1)
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class DeleteOp(BaseModel):
    """Delete exact text (first or all matches)."""
    op: Literal["delete"]
    text: str = Field(..., min_length=1)
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

# ───────────────────────────────────────────────────────────────────────────────
# New MVP ops
# ───────────────────────────────────────────────────────────────────────────────

class ReplaceFirstOp(BaseModel):
    op: Literal["replace_first"]
    find: str = Field(..., min_length=1)
    replace: str = Field(..., min_length=0)
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class InsertBeforeOp(BaseModel):
    op: Literal["insert_before"]
    text: str = Field(..., min_length=1)
    before: str = Field(..., min_length=1)
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class PrependOp(BaseModel):
    op: Literal["prepend"]
    text: str = Field(..., min_length=1)
    target: Optional[Targeting] = None  # target may scope prepend to a block
    model_config = ConfigDict(extra="forbid")

class AppendOp(BaseModel):
    op: Literal["append"]
    text: str = Field(..., min_length=1)
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class DeleteRegexOp(BaseModel):
    op: Literal["delete_regex"]
    pattern: str = Field(..., min_length=1)
    all_occurrences: bool = True
    flags_ignore_case: bool = False
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")
    @model_validator(mode="after")
    def _validate_regex(cls, v: "DeleteRegexOp"):
        try:
            re.compile(v.pattern)
        except re.error as e:
            raise ValueError(f"Invalid regex pattern: {e}") from e
        return v

class DeleteRangeOp(BaseModel):
    op: Literal["delete_range"]
    start: int = Field(..., ge=0)
    end: int = Field(..., ge=0)
    target: Optional[Targeting] = None  # range is measured in the (scoped) text
    model_config = ConfigDict(extra="forbid")
    @model_validator(mode="after")
    def _validate_range(cls, v: "DeleteRangeOp"):
        if v.end < v.start:
            raise ValueError("end must be >= start")
        return v

class DeleteBetweenOp(BaseModel):
    """Delete between markers (optionally including markers) in the (scoped) text."""
    op: Literal["delete_between"]
    start_marker: str = Field(..., min_length=1)
    end_marker: str = Field(..., min_length=1)
    include_markers: bool = False
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class WrapWithOp(BaseModel):
    """Surround target literal with prefix+suffix (first, nth, last or all)."""
    op: Literal["wrap_with"]
    target_text: str = Field(..., min_length=1)
    prefix: str = Field("", min_length=0)
    suffix: str = Field("", min_length=0)
    all_occurrences: bool = False
    target: Optional[Targeting] = None
    model_config = ConfigDict(extra="forbid")

class NormalizeWhitespaceOp(BaseModel):
    op: Literal["normalize_whitespace"]
    # options later if needed (e.g. keep double newlines)
    model_config = ConfigDict(extra="forbid")

# Line-wise helpers (voice friendly)
class SetLineTextOp(BaseModel):
    op: Literal["set_line_text"]
    line_index: int = Field(..., ge=0)
    text: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")

class InsertLineBeforeOp(BaseModel):
    op: Literal["insert_line_before"]
    line_index: int = Field(..., ge=0)
    text: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")

class InsertLineAfterOp(BaseModel):
    op: Literal["insert_line_after"]
    line_index: int = Field(..., ge=0)
    text: str = Field(..., min_length=0)
    model_config = ConfigDict(extra="forbid")

class DeleteLineOp(BaseModel):
    op: Literal["delete_line"]
    line_index: int = Field(..., ge=0)
    model_config = ConfigDict(extra="forbid")

# ───────────────────────────────────────────────────────────────────────────────
# Edit plan container
# ───────────────────────────────────────────────────────────────────────────────

class EditPlan(BaseModel):
    ops: List[
        Union[
            # existing
            ReplaceAllOp, RegexReplaceOp, InsertAtOp, InsertAfterOp, DeleteOp,
            # new
            ReplaceFirstOp, InsertBeforeOp, PrependOp, AppendOp,
            DeleteRegexOp, DeleteRangeOp, DeleteBetweenOp, WrapWithOp,
            NormalizeWhitespaceOp,
            SetLineTextOp, InsertLineBeforeOp, InsertLineAfterOp, DeleteLineOp,
        ]
    ]
    model_config = ConfigDict(extra="forbid")

## 🤖 Core Functions

**`_plan_edits(instruction, model)`**
- Appends user instruction to conversation
- Calls OpenAI API with `response_format=EditPlan` for structured output
- Returns parsed `EditPlan` object


### 🔄 Execution Flow
`_apply_plan(transcript, plan)`

The core executor.  
Iterates over all `ops` in the `EditPlan` and applies them using helper functions.

Each operation is wrapped in:   
```py 
updated = _apply_in_scope(updated, op.target, mutate)
```
This ensures that edits respect the defined scope.  

_____

`_apply_in_scope(full_text, tgt, mutator)`

_Scoping helper that decides what part of the text the_ `mutator()` _function should operate on._

**Supported scopes:**

`"global"` → the entire text.

`"line"` → a specific line.

`"line_range"` → multiple lines.

`"between_markers"` → text between start/end markers.
_____
`_replace_literal_scoped()`, `_find_all()`, `_split_lines_keepends()`, `_join_lines()`

Utility helpers for consistent behavior:

`_find_all()` → find all indices of a substring (with optional casefolding)

`replace_literal_scoped()` → literal replace with case/nth/all logic

`_split_lines_keepends()` → preserves line endings for line-based edits

`_join_lines()` → reassemble edited lines safely

In [ ]:
# | export

def _plan_edits(transcript: str, instruction: str, model: Optional[str] = None) -> EditPlan:
    """
    Append a user instruction, call the model with structured output, and return the parsed plan.
    """
    global _client, _current_model

    if _client is None:
        raise RuntimeError(
            "No AI client initialized. Set up OpenAI API key or start Ollama."
        )
    
    # Use provided model or fall back to current model
    model_to_use = model or _current_model

    messages = [
        {
            "role": "system",
            "content": (
                "You must output only valid EditPlan JSON.\n"
                "- Format: {\"ops\": [...]} — no text, no explanations.\n"
                "- Always use the most specific operation (e.g., delete_line, insert_line, replace_line).\n"
                "- line_index is 0-based (line 1 = 0).\n"
                "- Include 'target' only when fields like nth, all, scope, or ignore_case are explicitly needed.\n"
                "- Never guess values; omit optional fields unless clearly required.\n"
                "- The final output must be valid JSON that matches the EditPlan schema."
            )
        },
        {
            "role": "assistant",
            "content": f"Current text to edit:\n{transcript}",
        },
        {
            "role": "user",
            "content": f"Instruction: {instruction}"
        }
    ]

    completion = _client.chat.completions.parse(
        model=model_to_use,
        messages=messages,
        response_format=EditPlan,
        temperature=0
    )
    msg = completion.choices[0].message
    if msg.refusal:
        raise RuntimeError(f"Model refused: {msg.refusal}")
    return msg.parsed

def _split_lines_keepends(s: str) -> List[str]:
    return s.splitlines(keepends=True) or [""]

def _join_lines(lines: List[str]) -> str:
    return "".join(lines)

def _casefold(s: str) -> str:
    return s.casefold()

def _find_all(haystack: str, needle: str, ignore_case: bool) -> List[int]:
    if not needle:
        return []
    h = _casefold(haystack) if ignore_case else haystack
    n = _casefold(needle) if ignore_case else needle
    out, start = [], 0
    while True:
        idx = h.find(n, start)
        if idx == -1:
            return out
        out.append(idx)
        start = idx + len(n)

def _replace_literal_scoped(text: str, find: str, repl: str, ignore_case: bool, nth: Optional[Union[int, str]], all_matches: bool) -> str:
    if not find:
        return text
    if not ignore_case and all_matches and nth is None:
        return text.replace(find, repl)

    # fall back to manual walk for ignore_case/nth
    indices = _find_all(text, find, ignore_case)
    if not indices:
        return text
    if all_matches:
        # replace from left to right while adjusting offsets
        out = []
        last = 0
        src = text
        delta = 0
        for i, idx in enumerate(indices):
            idx += delta
            out.append(src[last:idx])
            out.append(repl)
            last = idx + len(find)
            delta += len(repl) - len(find)
        out.append(src[last:])
        return "".join(out)
    # nth or first/last
    if nth == "last":
        which = len(indices)
    else:
        which = 1 if nth is None else nth
    if which < 1 or which > len(indices):
        return text
    # compute real index considering earlier replacements = none (single replace)
    real_idx = indices[which - 1]
    return text[:real_idx] + repl + text[real_idx + len(find):]

def _apply_in_scope(full_text: str, tgt: Optional[Targeting], mutator) -> str:
    """
    Apply `mutator(scoped_text) -> new_scoped_text` within the region selected by `tgt`.
    If tgt is None or scope='global', apply to entire text.
    """
    if tgt is None or tgt.scope == "global":
        return mutator(full_text)

    if tgt.scope in ("line", "line_range"):
        lines = _split_lines_keepends(full_text)
        if tgt.scope == "line":
            if tgt.line_index is None or tgt.line_index >= len(lines):
                return full_text
            scoped = lines[tgt.line_index]
            lines[tgt.line_index] = mutator(scoped)
            return _join_lines(lines)

        # line_range
        start = min(tgt.line_start or 0, len(lines))
        end = min(tgt.line_end or len(lines), len(lines))
        if end <= start:
            return full_text
        scoped = "".join(lines[start:end])
        new_scoped = mutator(scoped)
        return _join_lines(lines[:start] + [new_scoped] + lines[end:])

    if tgt.scope == "between_markers":
        s_marker, e_marker = tgt.start_marker or "", tgt.end_marker or ""
        s_idx = full_text.find(s_marker)
        if s_idx == -1:
            return full_text
        e_idx = full_text.find(e_marker, s_idx + len(s_marker))
        if e_idx == -1:
            return full_text
        # selection boundaries (scope region)
        sel_start = s_idx if tgt.include_markers else s_idx + len(s_marker)
        sel_end   = e_idx + len(e_marker) if tgt.include_markers else e_idx
        scoped = full_text[sel_start:sel_end]
        new_scoped = mutator(scoped)
        return full_text[:sel_start] + new_scoped + full_text[sel_end:]

    return full_text

def _apply_plan(transcript: str, plan: EditPlan) -> str:
    updated = transcript

    for op in plan.ops:

        # ---------- literal replace ops ----------
        if op.op == "replace_all":
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                return _replace_literal_scoped(
                    s, op.find, op.replace, t.ignore_case, t.nth, t.all
                )
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "replace_first":
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                # force nth=1 if not otherwise specified
                nth = 1 if (t.nth is None and not t.all) else t.nth
                return _replace_literal_scoped(
                    s, op.find, op.replace, t.ignore_case, nth, False
                )
            updated = _apply_in_scope(updated, op.target, mutate)

        # ---------- regex replace ----------
        elif op.op == "regex_replace":
            flags = re.IGNORECASE if op.flags_ignore_case else 0
            pattern = re.compile(op.pattern, flags)
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                if t.all or t.nth is None:
                    return pattern.sub(op.replacement, s)
                # nth or last
                matches = list(pattern.finditer(s))
                if not matches:
                    return s
                if t.nth == "last":
                    m = matches[-1]
                    return s[:m.start()] + pattern.sub(op.replacement, s[m.start():m.end()]) + s[m.end():]
                if isinstance(t.nth, int) and 1 <= t.nth <= len(matches):
                    m = matches[t.nth - 1]
                    return s[:m.start()] + pattern.sub(op.replacement, s[m.start():m.end()]) + s[m.end():]
                return s
            updated = _apply_in_scope(updated, op.target, mutate)

        # ---------- inserts ----------
        elif op.op == "insert_at":
            def mutate(s: str) -> str:
                pos = max(0, min(op.position, len(s)))
                return s[:pos] + op.text + s[pos:]
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "insert_after":
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                indices = _find_all(s, op.after, t.ignore_case)
                if not indices:
                    return s
                idx = indices[-1] if t.nth == "last" else indices[(t.nth or 1) - 1]
                insert_pos = idx + len(op.after)
                return s[:insert_pos] + op.text + s[insert_pos:]
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "insert_before":
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                indices = _find_all(s, op.before, t.ignore_case)
                if not indices:
                    return s
                idx = indices[-1] if t.nth == "last" else indices[(t.nth or 1) - 1]
                return s[:idx] + op.text + s[idx:]
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "prepend":
            def mutate(s: str) -> str:
                return op.text + s
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "append":
            def mutate(s: str) -> str:
                return s + op.text
            updated = _apply_in_scope(updated, op.target, mutate)

        # ---------- deletions ----------
        elif op.op == "delete":
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                if t.all:
                    return _replace_literal_scoped(s, op.text, "", t.ignore_case, None, True)
                return _replace_literal_scoped(s, op.text, "", t.ignore_case, t.nth or 1, False)
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "delete_regex":
            flags = re.IGNORECASE if op.flags_ignore_case else 0
            pattern = re.compile(op.pattern, flags)
            def mutate(s: str) -> str:
                if op.all_occurrences:
                    return pattern.sub("", s)
                # single removal (first match)
                return pattern.sub("", s, count=1)
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "delete_range":
            def mutate(s: str) -> str:
                start = max(0, min(op.start, len(s)))
                end = max(start, min(op.end, len(s)))
                return s[:start] + s[end:]
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "delete_between":
            def mutate(s: str) -> str:
                s_idx = s.find(op.start_marker)
                if s_idx == -1: return s
                e_idx = s.find(op.end_marker, s_idx + len(op.start_marker))
                if e_idx == -1: return s
                if op.include_markers:
                    return s[:s_idx] + s[e_idx + len(op.end_marker):]
                else:
                    return s[:s_idx + len(op.start_marker)] + s[e_idx:]
            updated = _apply_in_scope(updated, op.target, mutate)

        # ---------- wrap/format ----------
        elif op.op == "wrap_with":
            def mutate(s: str) -> str:
                t = op.target or Targeting()
                needle = op.target_text
                if not needle:
                    return s
                if op.all_occurrences or t.all:
                    parts = re.split(f"({re.escape(needle)})", s)
                    for i in range(1, len(parts), 2):
                        parts[i] = f"{op.prefix}{parts[i]}{op.suffix}"
                    return "".join(parts)
                # nth/first/last single wrap with case-sensitive match (use Targeting.ignore_case if you want casefold logic)
                indices = _find_all(s, needle, t.ignore_case)
                if not indices:
                    return s
                idx = indices[-1] if t.nth == "last" else indices[(t.nth or 1) - 1]
                return s[:idx] + op.prefix + s[idx:idx+len(needle)] + op.suffix + s[idx+len(needle):]
            updated = _apply_in_scope(updated, op.target, mutate)

        elif op.op == "normalize_whitespace":
            tmp = updated.replace("\r\n", "\n").replace("\r", "\n")
            tmp = tmp.replace("\n", "⏎")
            tmp = re.sub(r"\s+", " ", tmp).strip()
            updated = "\n".join(part.strip() for part in tmp.split("⏎"))

        # ---------- line-wise ----------
        elif op.op == "set_line_text":
            lines = _split_lines_keepends(updated)
            if op.line_index < len(lines):
                # preserve the original line-ending (if any)
                ending = ""
                if lines[op.line_index] and lines[op.line_index].endswith(("\n", "\r\n")):
                    ending = "\n" if lines[op.line_index].endswith("\n") else ""
                lines[op.line_index] = op.text + ending
                updated = _join_lines(lines)

        elif op.op == "insert_line_before":
            lines = _split_lines_keepends(updated)
            idx = min(op.line_index, len(lines))
            newline = ("" if (idx>0 and lines[idx-1].endswith(("\n","\r\n"))) else "\n")
            lines.insert(idx, op.text + ("\n" if not op.text.endswith(("\n","\r\n")) else ""))
            updated = _join_lines(lines)

        elif op.op == "insert_line_after":
            lines = _split_lines_keepends(updated)
            idx = min(op.line_index, len(lines)-1)
            insert_at = idx + 1
            lines.insert(insert_at, op.text + ("\n" if not op.text.endswith(("\n","\r\n")) else ""))
            updated = _join_lines(lines)

        elif op.op == "delete_line":
            lines = _split_lines_keepends(updated)
            if op.line_index < len(lines):
                del lines[op.line_index]
                updated = _join_lines(lines)

    return updated

## 🔌 Public API

Functions for managing edit sessions and applying instructions.

In [ ]:
# | export
def get_current_model() -> str:
    """Get the currently selected model name."""
    return _current_model

def get_available_models() -> Dict[str, List[str]]:
    """Get available models from both OpenAI and Ollama."""
    models = {"openai": [], "ollama": []}
    
    # OpenAI models (hardcoded common ones)
    models["openai"] = [
        "gpt-4o", "gpt-4o-mini", "gpt-4", "gpt-4-turbo", 
        "gpt-3.5-turbo", "gpt-3.5-turbo-16k"
    ]
    
    # Ollama models (from installed models)
    ollama_models = _ollama_manager.list_models()
    models["ollama"] = [model["name"] for model in ollama_models]
    
    return models

def download_ollama_model(model_name: str) -> bool:
    """Download an Ollama model."""
    return _ollama_manager.pull_model(model_name)


def apply_instruction(transcript: str, instruction: str) -> str:
    """Apply an instruction to the current transcript and return the updated text."""
    plan = _plan_edits(transcript, instruction)
    transcript = _apply_plan(transcript, plan)
    return transcript, plan

## Example using the Public API

In [ ]:
#| eval: false
transcript = (
    "Hello everyone.\n"
    "Today i will teach you about using AI in python.\n"
    "Lets start by looking into API's.\n"
)
instruction = "Add a line before the last one saying. Open your books on page 100"
transcript, plan = apply_instruction(instruction)
print(transcript)
print(f"Plan: {plan.model_dump_json(indent=2)}")